In [1]:
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

In [30]:
url_client = 'TOOL_CLIENT.csv'
url_sales = 'TOOL_SALES.csv'
sep = ','

df_client = pd.read_csv(url_client, sep=sep)

df_sales = pd.read_csv(url_sales, sep=sep)

In [31]:
df_client.head()

,CLIENT_ID,CLIENT_CREATE DATE,REGION,TRADE SECTOR,N_EMPLOYEES,ECONOMIC_POT,ECO_POT_CLASS,RISK_CAT
0,9306,2005-11-15 00:00:00,BZ,11000,6,8659.81,D,3d
1,939,2005-11-15 00:00:00,LE,15500,2,681.26,E,3d
2,8321,2005-11-15 00:00:00,LE,15500,2,681.26,E,T8
3,4174,2005-11-15 00:00:00,BZ,15400,1,494.45,E,3d
4,12765,2005-11-15 00:00:00,BZ,15400,1,494.45,E,3b


In [32]:
df_sales.head()

,YYYYMM,ITEM_ID,FLG_TOOL,SALES_CHANNEL,NET,UNIT,FAMILY_CODE,GROUP_CODE,CLIENT_ID,CANCELLED
0,201701,54,0,B,24.78,P,XBAM2HA,XBAM2HA0201,15674,NaN
1,201701,44,0,B,34.62,P,XBAM2HA,XBAM2HA0201,62486,NaN
2,201701,47,0,D,46.45,P,XBAM2HA,XBAM2HA0201,60149,NaN
3,201701,52,0,B,62.15,P,XBAM2HA,XBAM2HA0201,42371,NaN
4,201701,56,0,B,35.80,P,XBAM2HA,XBAM2HA0201,13144,NaN


In [33]:
df_client.describe(include='all')

,CLIENT_ID,CLIENT_CREATE DATE,REGION,TRADE SECTOR,N_EMPLOYEES,ECONOMIC_POT,ECO_POT_CLASS,RISK_CAT
count,93257.000000,93257,91276,93257.000000,93257.000000,93257.000000,93257,93257
unique,NaN,4159,121,NaN,NaN,NaN,2,20
top,NaN,2005-11-15 00:00:00,RM,NaN,NaN,NaN,E,3d
freq,NaN,14194,4768,NaN,NaN,NaN,57725,30918
mean,46629.000000,NaN,NaN,17519.569577,5.428483,4942.023531,NaN,NaN
std,26921.121364,NaN,NaN,8769.651973,48.498675,2993.796073,NaN,NaN
min,1.000000,NaN,NaN,10000.000000,1.000000,0.000000,NaN,NaN
25%,23315.000000,NaN,NaN,11000.000000,1.000000,2264.230000,NaN,NaN
50%,46629.000000,NaN,NaN,13500.000000,3.000000,4528.530000,NaN,NaN
75%,69943.000000,NaN,NaN,21100.000000,5.000000,7528.520000,NaN,NaN


In [34]:
df_sales.describe(include='all')

,YYYYMM,ITEM_ID,FLG_TOOL,SALES_CHANNEL,NET,UNIT,FAMILY_CODE,GROUP_CODE,CLIENT_ID,CANCELLED
count,2.050449e+06,2.050449e+06,2.050449e+06,2050449,2.050449e+06,2050449,2050449,2050449,2.050449e+06,102710
unique,NaN,NaN,NaN,4,NaN,4,762,2206,NaN,1
top,NaN,NaN,NaN,B,NaN,P,XAEP3CC,XAEP3CC0204,NaN,X
freq,NaN,NaN,NaN,1451851,NaN,2035647,135774,75960,NaN,102710
mean,2.018903e+05,6.152528e+03,3.593223e-01,NaN,1.168498e+02,NaN,NaN,NaN,4.085241e+04,NaN
std,1.428000e+02,4.524401e+03,4.798019e-01,NaN,2.273342e+02,NaN,NaN,NaN,2.428800e+04,NaN
min,2.017010e+05,1.000000e+00,0.000000e+00,NaN,-1.313297e+04,NaN,NaN,NaN,1.000000e+00,NaN
25%,2.018010e+05,2.470000e+03,0.000000e+00,NaN,1.368000e+01,NaN,NaN,NaN,1.997500e+04,NaN
50%,2.019030e+05,4.815000e+03,0.000000e+00,NaN,4.909000e+01,NaN,NaN,NaN,4.046800e+04,NaN
75%,2.020090e+05,1.007700e+04,1.000000e+00,NaN,1.357800e+02,NaN,NaN,NaN,5.997900e+04,NaN
